In [1]:
#libraries
from ortools.linear_solver import pywraplp as glp

In [2]:
#inputs
max_lab = 100.0 #max labour hour each week
machine_lab = [2.0,1.0] #2 hours of labour to operate 1 hr of machine 1, 1 hours of labour to operate 1 hr of machine 2
product = ['product 1','product 2','product 3']
mach_hr = [[0.5,1.0],[2.0,1.0],[0.75,0.5]] #Machine 1 & 2 hour per unit for each product
unit_profit = [30.0,50.0,20.0] #for each product
max_prod1 = 0.5 #max percent of product 1 in total product
min_prod3 = 0.2 #min percent of product 3 in total product
mach_limit = 40.0 #In a typical week, 40 hours are available on each machine.

In [3]:
#Create model
model = glp.Solver('Better',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [4]:
#Decision variables
dvar = list(range(len(product)))
for i in range(len(product)):
    dvar[i]=model.NumVar(0,model.infinity(),product[i])

In [5]:
#Objective Function
TotProfit = model.Objective()
TotProfit.SetMaximization()
for i in range(len(dvar)):
    TotProfit.SetCoefficient(dvar[i],unit_profit[i])

In [6]:
#Constraints
constr = list(range(len(machine_lab)+3))
#2 constraints on machine hour 40 per week
for m in range(len(machine_lab)):
    constr[m] = model.Constraint(-model.infinity(),mach_limit,'Machine %d hour limit' % (m+1))
    for p in range(len(dvar)):
        constr[m].SetCoefficient(dvar[p],mach_hr[p][m])
#constraint on max labour
temp=mach_hr
constr[2] = model.Constraint(-model.infinity(), max_lab, 'Max labour')
for p in range(len(product)):
    for m in range(len(machine_lab)):
        temp[p][m] = mach_hr[p][m]*machine_lab[m]
    prod_lab = sum(temp[p]) #labour in hour for each product
    constr[2].SetCoefficient(dvar[p],prod_lab)
#constraint on max percent of product 1 in total product 
constr[3] = model.Constraint(-model.infinity(), 0, 'Max % prod 1')
for p in range(len(product)):
    if product[p] == 'product 1': 
        constr[3].SetCoefficient(dvar[p],(1.0 - max_prod1))
    else: 
        constr[3].SetCoefficient(dvar[p],(-max_prod1))
#constraint on min percent of product 3 in total product 
constr[4] = model.Constraint(0, model.infinity(), 'Min % prod 3')
for p in range(len(product)):
    if product[p] == 'product 3': 
        constr[4].SetCoefficient(dvar[p],(1.0 - min_prod3))
    else: 
        constr[4].SetCoefficient(dvar[p],(-min_prod3))

In [7]:
#Solve model and display result
statusls = ['Optimal','Feasible','Infeasible','Unbounded','Abnormal','Not Solved']
status = model.Solve()
print('Solution status:', statusls[status])
print('Number of variables:', model.NumVariables())
print('Number of constraints:',model.NumConstraints())
print('Optimal Solution')
print('Total Profit: $%.2f' % TotProfit.Value())
for p in range(len(product)):
    print('%s = %.0f units' % (product[p],dvar[p].solution_value()))

Solution status: Optimal
Number of variables: 3
Number of constraints: 5
Optimal Solution
Total Profit: $1250.00
product 1 = 25 units
product 2 = 0 units
product 3 = 25 units


In [8]:
#Display Constraints info
LHS = model.ComputeConstraintActivities()
for i in range(2):
    print('Machine %d hours: %.2f %.2f %.2f' % ((i+1),LHS[i],mach_limit,constr[i].dual_value()))
print('Total labour hours: %.2f %.2f %.2f' % (LHS[2],max_lab,constr[2].dual_value()))
totalprod = 0
for i in range(len(product)):
    totalprod += dvar[i].solution_value()
print('Percent of Product 1: %.1f%% %.1f%%' % ((dvar[0].solution_value()/totalprod)*100,max_prod1*100))
print('Percent of Product 3: %.1f%% %.1f%%' % ((dvar[2].solution_value()/totalprod)*100,min_prod3*100))

Machine 1 hours: 31.25 40.00 -0.00
Machine 2 hours: 37.50 40.00 -0.00
Total labour hours: 100.00 100.00 12.50
Percent of Product 1: 50.0% 50.0%
Percent of Product 3: 50.0% 20.0%


In [9]:
#if labour capacity increases by 20 hr
#Modify the max labour constraint
temp=mach_hr
new_max_lab= max_lab+20
constr[2].SetUb(new_max_lab)

In [10]:
#Solve model and display result
statusls = ['Optimal','Feasible','Infeasible','Unbounded','Abnormal','Not Solved']
status = model.Solve()
print('Solution status:', statusls[status])
print('Number of variables:', model.NumVariables())
print('Number of constraints:',model.NumConstraints())
print('Optimal Solution')
print('Total Profit: $%.2f' % TotProfit.Value())
for p in range(len(product)):
    print('%s = %.0f units' % (product[p],dvar[p].solution_value()))

Solution status: Optimal
Number of variables: 3
Number of constraints: 5
Optimal Solution
Total Profit: $1440.00
product 1 = 24 units
product 2 = 8 units
product 3 = 16 units


In [13]:
#Display Constraints info
LHS = model.ComputeConstraintActivities()
for i in range(2):
    print('Machine %d hours: %.2f %.2f %.2f' % ((i+1),LHS[i],mach_limit,constr[i].dual_value()))
print('Total labour hours: %.2f %.2f %.2f' % (LHS[2],new_max_lab,constr[2].dual_value()))
totalprod = 0
for i in range(len(product)):
    totalprod += dvar[i].solution_value()
print('Percent of Product 1: %.1f%% %.1f%%' % ((dvar[0].solution_value()/totalprod)*100,max_prod1*100))
print('Percent of Product 3: %.1f%% %.1f%%' % ((dvar[2].solution_value()/totalprod)*100,min_prod3*100))

Machine 1 hours: 40.00 40.00 -0.00
Machine 2 hours: 40.00 40.00 12.00
Total labour hours: 120.00 120.00 8.00
Percent of Product 1: 50.0% 50.0%
Percent of Product 3: 33.3% 20.0%


At the new labor capacity, profit increased from the previous value of 1250 to 1440 and we maximize the capacity of our machines. Thus, we should use this new capacity.